In [12]:
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
import json

In [5]:
THRESHOLD=3
DATASET="musicalInstrument"

In [6]:
rating_df=pd.DataFrame([json.loads(line) for line in open('../data/'+DATASET+'/raw/Musical_Instruments_5.json','r')])
item_meta_df=pd.DataFrame([json.loads(line) for line in open('../data/'+DATASET+'/raw/meta_Musical_Instruments.json','r')])

In [13]:
entity_id2index = dict()
item_index_to_id = dict()

def form_item_index_to_id():
    # rating_df=pd.read_csv('../data/'+DATASET+'/raw/Musical_Instruments.csv',header=None)
   
    # rating_df.columns=['item_id','user_id','rating','timestamp']
    # display(rating_df.head())
    items = rating_df['asin'].unique()
    # print(rating_df['reviewerID'].unique().shape)
    for i in range(len(items)):
        entity_id2index[items[i]] = i
        item_index_to_id[i] = items[i]
form_item_index_to_id()
    

In [14]:
user_index_old2new = dict()

def convert_rating():
    
    print('preparing rating file ...')
    item_set = set(entity_id2index.values())
    user_pos_ratings = dict()
    user_neg_ratings = dict()

    def get_user_pos_neg_ratings(x):
        if float(x['overall']) >= THRESHOLD:
            if x['reviewerID'] not in user_pos_ratings:
                user_pos_ratings[x['reviewerID']] = set()
            user_pos_ratings[x['reviewerID']].add(entity_id2index[x['asin']])
        else:
            if x['reviewerID'] not in user_neg_ratings:
                user_neg_ratings[x['reviewerID']] = set()
            user_neg_ratings[x['reviewerID']].add(entity_id2index[x['asin']])

    rating_df.apply(get_user_pos_neg_ratings, axis=1)
    # print(len(user_pos_ratings))
    # print(len(user_neg_ratings))
    # for line in open(file, encoding='utf-8').readlines()[1:]:
    #     array = line.strip().split(SEP[DATASET])

        # # remove prefix and suffix quotation marks for BX dataset
        # if DATASET == 'book':
        #     array = list(map(lambda x: x[1:-1], array))

        # item_index_old = array[1]
        # if item_index_old not in item_index_old2new:  # the item is not in the final item set
        #     continue
        # item_index = item_index_old2new[item_index_old]

        # user_index_old = int(array[0])

        # rating = float(array[2])
        # if rating >= THRESHOLD[DATASET]:
        #     if user_index_old not in user_pos_ratings:
        #         user_pos_ratings[user_index_old] = set()
        #     user_pos_ratings[user_index_old].add(item_index)
        # else:
        #     if user_index_old not in user_neg_ratings:
        #         user_neg_ratings[user_index_old] = set()
        #     user_neg_ratings[user_index_old].add(item_index)

    print('converting rating file ...')
    writer = open('../data/' + DATASET + '/ratings_final.txt', 'w', encoding='utf-8')
    user_cnt = 0
    
    for user_index_old, pos_item_set in tqdm(user_pos_ratings.items()):
        if len(pos_item_set) < 5:
            continue
        if user_index_old not in user_index_old2new:
            user_index_old2new[user_index_old] = user_cnt
            user_cnt += 1
        user_index = user_index_old2new[user_index_old]

        for item in pos_item_set:
            writer.write('%d\t%d\t1\n' % (user_index, item))
        unwatched_set = item_set - pos_item_set
        if user_index_old in user_neg_ratings:
            unwatched_set -= user_neg_ratings[user_index_old]
        for item in np.random.choice(list(unwatched_set), size=len(pos_item_set), replace=False):
            writer.write('%d\t%d\t0\n' % (user_index, item))
    writer.close()
    print('number of users: %d' % user_cnt)
    print('number of items: %d' % len(item_set))


convert_rating()

preparing rating file ...
converting rating file ...


100%|██████████| 27492/27492 [00:21<00:00, 1276.83it/s]

number of users: 22717
number of items: 10620


In [15]:
relation_id2index = dict()
entity_cnt = len(entity_id2index)

def convert_kg():
    print('converting kg file ...')
    
    relation_cnt = 0

    writer = open('../data/' + DATASET + '/kg_final.txt', 'w', encoding='utf-8')

    def add_links(x):
        global entity_cnt
        if x['asin'] not in entity_id2index:
            return
        item_index = entity_id2index[x['asin']]
        # user_index = user_index_old2new[x['reviewerID']]
        for relation_index, relation in enumerate(["category" , "also_buy", "feature"]):
            for tail in x[relation]:
                if tail not in entity_id2index:
                    entity_id2index[tail] = entity_cnt
                    entity_cnt += 1

                tail_index = entity_id2index[tail]
                writer.write('%d\t%d\t%d\n' % (item_index, relation_index, tail_index))
                writer.write('%d\t%d\t%d\n' % (tail_index, relation_index + 4, item_index))
        tail=x['brand']
        if tail not in entity_id2index:
            entity_id2index[tail] = entity_cnt
            entity_cnt += 1
        tail_index = entity_id2index[tail]
        writer.write('%d\t%d\t%d\n' % (item_index, 3, tail_index))
        writer.write('%d\t%d\t%d\n' % (tail_index, 7, item_index))
    item_meta_df.apply(add_links, axis=1)


    # files = []
    # if DATASET == 'movie':
    #     files.append(open('../data/' + DATASET + '/kg_part1_rehashed.txt', encoding='utf-8'))
    #     files.append(open('../data/' + DATASET + '/kg_part2_rehashed.txt', encoding='utf-8'))
    # else:
    #     files.append(open('../data/' + DATASET + '/kg_rehashed.txt', encoding='utf-8'))

    # for file in files:
    #     for line in file:
    #         array = line.strip().split('\t')
    #         head_old = array[0]
    #         relation_old = array[1]
    #         tail_old = array[2]

    #         if head_old not in entity_id2index:
    #             entity_id2index[head_old] = entity_cnt
    #             entity_cnt += 1
    #         head = entity_id2index[head_old]

    #         if tail_old not in entity_id2index:
    #             entity_id2index[tail_old] = entity_cnt
    #             entity_cnt += 1
    #         tail = entity_id2index[tail_old]

    #         if relation_old not in relation_id2index:
    #             relation_id2index[relation_old] = relation_cnt
    #             relation_cnt += 1
            # relation = relation_id2index[relation_old]

            # writer.write('%d\t%d\t%d\n' % (head, relation, tail))

    writer.close()
    print('number of entities (containing items): %d' % entity_cnt)
    print('number of relations: %d' % relation_cnt)

convert_kg()

converting kg file ...
number of entities (containing items): 90358
number of relations: 0


In [43]:
item_meta_df
# category(list), brand , also_buy(list), feature(list)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Musical Instruments, Drums & Percussion, Hand...",,[Cricket Rubbing the spine with the wooden sti...,,Wooden Percussion 2 Piece Set of 3 Inch Cricke...,"[B00NP8GYVS, B00NP80XMO, B00NP8M098]",,WADSUWAN SHOP,"[Wood percussion, Owl whistle*, Includes woode...","[>#141,729 in Musical Instruments (See Top 100...",[],Musical Instruments,,"December 2, 2013",,0000989983,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Musical Instruments, Drums & Percussion, Hand...",,[Frog - Rubbing its spine with the wooden stic...,,"Wooden Percussion 3 Piece Set Frog, Cricket an...","[B00NP8GYVS, B00NP80XMO, B01MY48HK5, B00AZZ1AJ...",,WADSUWAN SHOP,"[Wood percussion, Small 3 inches, Creates orig...","[>#1,622 in Musical Instruments (See Top 100 i...",[],Musical Instruments,,"December 2, 2013",$0.91,0000098906,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Musical Instruments, Instrument Accessories, ...",,[Vivaldi's famous set of four violin concertos...,,Hal Leonard Vivaldi Four Seasons for Piano (Or...,[],,Hal Leonard,"[., ., .]","[>#330,653 in Musical Instruments (See Top 100...",[],Musical Instruments,,"May 10, 2011",$62.93,0041291905,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
3,[],,"[The Turn of the Screw (op. 54) vocal score, p...",,The Turn of the Screw (vocal score),"[0486266842, 0793507669, 0393008789, 142341280...",,Boosey &amp; Hawkes,[],"[>#86,354 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"May 23, 2007",$107.79,0060015500,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,[],,[],,Suite for Organ (including the Trumpet Volunta...,[],,,[],"[>#482,025 in Musical Instruments (See Top 100...",[],Musical Instruments,,"February 8, 2013",,0193757710,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120305,"[Musical Instruments, Amplifiers & Effects, Gu...",,[This packages includes:<br> <br> 10 pcs 3pdt ...,,10 pcs 3pdt Stomp Footswitch incl PCB incl met...,[],,3pdt,[5 pcs 9 Pole Footswitch for True Bypass Switc...,"[>#23,334 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"June 25, 2016",$7.99,B01HJDOF2Y,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
120306,"[Musical Instruments, Amplifiers & Effects, Gu...",,[This packages includes:<br> <br> 5 pcs 3pdt T...,,"5 pcs 3pdt Stomp Footswitch incl. PCB, metal w...","[B015334J9G, B01L6GU858, B003HLW14W, B012CF181...",,3pdt,[5 pcs 9 Pole Footswitch for True Bypass Switc...,"[>#40,026 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"June 25, 2016",$0.91,B01HJDJ1PA,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
120307,"[Musical Instruments, Drums & Percussion, Hand...",,[These high quality clear crystal singing bowl...,,Optical Clear Quartz Crystal Singing Bowl Note...,[],,Crystal Energy Bowls,[High quality clear crystal singing bowl is wi...,"[>#481,984 in Musical Instruments (See Top 100...",[],Musical Instruments,,"June 25, 2016",,B01HJEFFTK,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
120308,"[Musical Instruments, Drums & Percussion, Hand...",,[Frog - Rubbing its spine with the wooden stic...,,2x Deluxe Small 2&quot; Wood Frog Guiro Rasp -...,[],,RA,"[Wood percussion, Small 2 inches, Packs of 2 P...","[>#76,917 in Musical Instruments (See Top 100 ...","[B00NP80XMO, B00NP8GYVS, 0000098906, B0149IWDS...",Musical Instruments,"class=""a-bordered a-horizontal-strip